<a href="https://colab.research.google.com/github/freison/DeepLearning/blob/master/Lab3_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3: Model-Free Reinforcement Learning

## Part 1: Cartpole

In [2]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay scikit-video > /dev/null 2>&1

import tensorflow as tf
tf.enable_eager_execution()

import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
import time

# Download the class repository
! git clone https://github.com/aamini/introtodeeplearning_labs.git  > /dev/null 2>&1
% cd introtodeeplearning_labs 
! git pull
% cd ..

import introtodeeplearning_labs as util

/content/introtodeeplearning_labs
Already up to date.
/content


### 1.1 Define and inspect the environment

In [3]:
'''In order to model our enviroment we will use OpenAI Gym, RL algorithms in many aspects are random,
   So we are going to use a seed.'''
env = gym.make("CartPole-v0")
env.seed(1) # Reproducible, since RL has high variance.

[1L]